# Simulated Method of Moments

## Introduction

The simulated method of moments (SMM) is some kind of a sledgehammer approach to estimate model parameters. It is a deviation from the general method of moments (GMM) which does not require that the function mapping parameters to moments has a closed-form solution. Thus, we want to minimize the distance between the actual moments and the simulated moments implied by the model parameters. Mathematically, we want to minimize

$$
    \min_\theta \; (m(\theta) - m^*)' W (m(\theta) - m^*)
$$

where $\theta$ is the parameter vector, $m(\theta)$ are the simulated moments implied by the parameters, $m^*$ are the moments from the data and $W$ is the weighting matrix.

## The weighting matrix

In [1]:
%matplotlib inline

import numpy as np
import pandas as pd
import respy as rp
import matplotlib.pyplot as plt

from estimagic.optimization.optimize import minimize
from respy.pre_processing.model_processing import process_params_and_options


CHOICES = ["blue_collar", "military", "white_collar", "school", "home"]

## Application

After learning about this new technique, we want to employ it. Usually, you have a dataset at hand on which to estimate the model parameters. Here, we try something different.

As ``respy`` is designed to unify research on discrete choice dynamic programming models, human capital accumulation and occupational choice, we try to replicate older models. These exercises help to learn about new model features and serve as a reference point for the outcomes. The problem is that we usually do not have access to the data of the paper and cannot estimate parameters via maximum likelihood estimation. Luckily, papers report more and less detailed summary statistics of the data. These statistics can be used to estimate parameters by simulated method of moments.

In the following, we will estimate the model parameters of Keane and Wolpin (2000).

In [2]:
params, options = rp.get_example_model("kw_2000", with_data=False)
options["n_periods"] = 11

### 1. Collect the actual moments

The first step is to collect the moments from the actual data. There are various tables in the paper which present usable statistics. The function below combines all the statistics into a single moments vector.

In [44]:
def calculate_moments_from_actual_data():
    # Add choice probabilties. First whites, then blacks.
    choice_probs_whites = pd.read_csv(
        rp.config.TEST_RESOURCES_DIR / "kw_2000_table_1_whites_choice_probabilities.csv", index_col="age"
    ).reindex(columns=CHOICES).to_numpy().ravel()
    choice_probs_blacks = pd.read_csv(
        rp.config.TEST_RESOURCES_DIR / "kw_2000_table_2_blacks_choice_probabilities.csv", index_col="age"
    ).reindex(columns=CHOICES).to_numpy().ravel()
    
    # Add mean wages.
    wages_whites = pd.read_csv(
        rp.config.TEST_RESOURCES_DIR / "kw_2000_table_3_wage_fit_whites.csv", index_col="age"
    ).reindex(columns=CHOICES[:3]).to_numpy().ravel()
    wages_blacks = pd.read_csv(
        rp.config.TEST_RESOURCES_DIR / "kw_2000_table_3_wage_fit_blacks.csv", index_col="age"
    ).reindex(columns=CHOICES[:3]).to_numpy().ravel()
    
    education = pd.read_csv(
        rp.config.TEST_RESOURCES_DIR / "kw_2000_table_5_school_attainment.csv", index_col=0
    ).drop(columns="age").to_numpy().ravel()
    
    moments = np.hstack((
        choice_probs_whites,
        choice_probs_blacks,
        wages_whites,
        wages_blacks,
        education,
    ))

    return moments    

In [45]:
moments = calculate_moments_from_actual_data()

### 2. Define function to calculate moments.

In [12]:
simulate = rp.get_simulate_func(params, options)
df = simulate(params)

In [49]:
def calculate_moments_from_simulated_data(df):
    df = df.query("Period <= 10")
    
    choice_probs = df.groupby(["Black", "Period"]).Choice.value_counts(
        normalize=True).unstack().reindex(columns=CHOICES).to_numpy().ravel()
    
    wages = (
        df.query("1 <= Period <= 10")
        .groupby(["Black", "Period", "Choice"])
        .Wage.mean()
        .unstack()
        .reindex(columns=CHOICES[:3])
        .to_numpy()
        .ravel()
    )

    sub = df.query("Period == 7").copy()
    sub["education_levels"] = pd.cut(sub.Experience_School, bins=[0, 11, 12, 15, 20])
    shares = sub.groupby("Black").education_levels.value_counts(normalize=True).unstack().T
    education = shares.append(sub.groupby("Black").Experience_School.mean()).to_numpy().ravel()
    
    moments = np.hstack((
        choice_probs,
        wages,
        education
    ))
    
    return moments

In [50]:
moments_ = calc_moments(df)

### 2. Choose a weighting matrix.

#### 2.1. Bootstrapping the variances of moments

In [5]:
def bootstrap_weighting_matrix(df, calc_moments, n_bootstraps, n_individuals):
    identifiers = df.Identifier.unique()
    
    moments = []
    for _ in range(n_bootstraps):
        idents = np.random.choice(identifiers, size=n_individuals, replace=True)
        sample = df.loc[df.Identifier.isin(idents)]
        bootstrapped_moments = calc_moments(sample)
        moments.append(bootstrapped_moments)
    moments = np.column_stack(moments)
    
    standard_deviations = moments.std(axis=1)
    standard_deviations = np.where(
        standard_deviations == 0, 10, standard_deviations
    )
    
    weighting_matrix = np.diag(standard_deviations)
        
    return weighting_matrix

#### 2.2. Two-step estimation

1. Start with the identity matrix and estimate optimal parameters.
2. Use the optimal parameters to calculate the moment errors and use them to compute another weighting matrix.
3. Rerun another estimation with the new weighting matrix.

In [ ]:
smm = rp.get_smm_func(params, options, moments, calculate_moments_from_simulated_data)

In [ ]:
params["group"] = params.index.get_level_values('category')

In [ ]:
constr = rp.get_parameter_constraints("kw_2000")

In [ ]:
results, params = minimize(
    smm,
    params,
    "scipy_L-BFGS-B",
    algo_options={"maxfun": 1000},
    constraints=constr,
)

In [ ]:
params.to_csv("kw_2000_W.csv")

In [52]:
def recover_weighting_matrix(params, smm):
    simulate = smm.keywords["simulate"]
    calc_moments = smm.keywords["calc_moments"]
    
    df = simulate(params)
    estimated_moments = calc_moments(df)
    moments_error = estimated_moments - moments
    weighting_matrix = np.outer(moments_error, moments_error)
    
    return weighting_matrix

In [ ]:
weighting_matrix = recover_weighting_matrix(params, smm)

### 3. Estimate.

In [7]:
smm = rp.get_smm_func(params, options, moments, calculate_moments_from_simulated_data, weighting_matrix)

In [8]:
params["group"] = params.index.get_level_values('category')

In [9]:
results, params = minimize(
    smm,
    params,
    "scipy_L-BFGS-B",
    constraints=constr,
)